In [5]:
import os
os.chdir('../../src/')
from visualization.SIR_Plot import make_SIR_graph, make_beds_graph

In [6]:
os.chdir('models')
import simulate_pandemic as simp
from policies import policies, costs
from MCTS import mcts, treeNode
from CMDP import CovidState

In [7]:
from tqdm import tqdm
import numpy as np

In [8]:
def run_full_mcst(rolloutPolicy='rolloutPolicy', iterationLimit=100, sim_rounds=10, n_jobs = 6, step_size = 7, days = 210):

    pop_matrix = simp.init_infection(.0001)
    data = []
    actions = []

    for day in tqdm(range(1, days+1)):
        #if less than 20% still susceptible, break simulation
        if pop_matrix[np.where(pop_matrix[:,1] == -1)].shape[0] > pop_matrix.shape[0]*.9: break            
            
        
        # Choose a new policy at each week
        if day % step_size == 1:                    
            tree = mcts(iterationLimit=iterationLimit, sim_rounds=sim_rounds, step_size=step_size, 
                        n_jobs=n_jobs, rolloutPolicy=rolloutPolicy, pop_matrix=pop_matrix)

            root = treeNode(CovidState(actions=[], day=day), parent=None)
            action, best_node = tree.search(root)

            actions.append(action)
            restrictions = policies[action]

        pop_matrix = simp.spread_infection(pop_matrix, restrictions, day)
        pop_matrix = simp.lambda_leak_expose(pop_matrix, day)
        pop_matrix = simp.update_population(pop_matrix)

        data.append(np.array(sorted(pop_matrix,key=lambda x: x[0]))[:,1]) 
    
    return data, actions, tree

In [9]:
import pickle as pkl

for i in range(1,2):

    data, actions, tree = run_full_mcst(iterationLimit=50 , sim_rounds=12, days=350, step_size=7, n_jobs=6)

    with open(f'25_06_{i}.pkl', 'wb') as f:
        pkl.dump((data,actions), f)

 40%|███████████████████████████████▏                                              | 140/350 [49:22<1:14:03, 21.16s/it]


KeyboardInterrupt: 

In [11]:
make_beds_graph(data, actions, 7, 'low_capacity_23_06_5')

In [16]:
tree.root.children['Light Quarantine']

In [17]:
tree.root.children['Light Quarantine'].children['Light Quarantine']

In [22]:
tree.root.children['Light Quarantine'].children['Light Quarantine'].children['Light Quarantine']

{'Lockdown': <MCTS.treeNode at 0x24eb4db6348>,
 'Hard Quarantine': <MCTS.treeNode at 0x24eb4e79ac8>,
 'Light Quarantine': <MCTS.treeNode at 0x24eaedc2848>,
 'Social Distancing': <MCTS.treeNode at 0x24eb4cfeec8>,
 'Unrestricted': <MCTS.treeNode at 0x24eb4d74848>}

In [27]:
tree.root.children['Light Quarantine'].children['Light Quarantine'].children['Light Quarantine'].state.actions

['Light Quarantine', 'Light Quarantine', 'Light Quarantine']

In [28]:
tree.root.children['Light Quarantine'].children['Light Quarantine'].children[c]

In [35]:
tree.root.children

{'Lockdown': <MCTS.treeNode at 0x24eb516d0c8>,
 'Hard Quarantine': <MCTS.treeNode at 0x24eb4f38788>,
 'Light Quarantine': <MCTS.treeNode at 0x24eb58ff208>,
 'Social Distancing': <MCTS.treeNode at 0x24eb5135f48>,
 'Unrestricted': <MCTS.treeNode at 0x24eb5664c88>}